In [1]:
import numpy as np
import shapley 
import RDG_gaussian
import RDG_interventional
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
def f(xs):
    if type(xs).__module__ == np.__name__:
        
        #xs[:,0] + np.exp(xs[:,1] / (2 + xs[:,4])) + 2*(xs[:,2] * xs[:,3]) + np.sin(xs[:,5] + xs[:,6])
        return xs[:,0] + 0.5*xs[:,1] + 0.8*xs[:,3] + 0.2*xs[:,2] + 0.5*xs[:,4] 
    else:
        #xs[0] + np.exp(xs[1] / (2 + xs[4])) + 2*(xs[2] * xs[3]) + np.sin(xs[5] + xs[6])
        return xs[0] + 0.5*xs[1] + 0.8*xs[3] + 0.2*xs[2] + 0.5*xs[4] 

In [3]:
def MAE():
    X1 = []
    for i in range(1000):
        x1 = []
        pop = random.uniform(-1,1)
        x1.append(pop)
        x1.append(2*pop)
        for i in range(0,3):
            x1.append(random.uniform(-1,1))
        x1.append(5*pop)
        X1.append(x1)

    X2 = []
    for i in range(1000):
        x2 = []
        pop = random.uniform(-1,1)
        x2.append(pop)
        x2.append(2*pop)
        for i in range(0,3):
            x2.append(random.uniform(-1,1))
        x2.append(5*pop)
        X2.append(x2)

    X = np.concatenate([np.asarray(X1),np.asarray(X2)])
    Y = f(X)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    from sklearn.ensemble import RandomForestRegressor    
    from sklearn.linear_model import LinearRegression
    #clf = LinearRegression().fit(X_train, y_train)
    clf = RandomForestRegressor(max_depth=10, random_state=0).fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    cov = np.cov(X.T)
    means = X.mean(axis=0)
    overall_ss = []
    overall_s_i = []
    overall_s_c = []
    for a in range(0,10):
        print(a)
        print(X_train.shape)
        x = X_test[random.randint(0,200)]
       
        ss_atts,seps,nonseps = RDG_gaussian.RDG(clf.predict,np.asarray(x),means,cov,[-1,-2,-1,-1,-1,-5],[1,2,1,1,1,5],0.15,X_train)
        s_i_atts = shapley.shapley_int(clf.predict,np.asarray(x),np.asarray(means))
        s_c_atts = shapley.shapley_cond_gaussian(clf.predict,np.asarray(x),X)
       
        print(ss_atts)
        ground_truth = {}
        ground_truth[0] = x[0]
        ground_truth[1] = 0.5*x[1]
        ground_truth[2] = 0.2*x[2]
        ground_truth[3] = 0.8*x[3]
        ground_truth[4] = 0.5*x[4]
        ground_truth[5] = 0
        ss_error = []
        sorted_dict = np.zeros(len(x))
        for i in range(0,len(x)):
            if sorted_dict[i] == 0:
                u_index = 0
                for u in ss_atts.keys():
                    if i in list(u):
                        gt = 0
                        for j in list(u):
                            gt += ground_truth[j]
                            sorted_dict[j] = 1
                        ss_error.append(np.abs(gt - list(ss_atts.values())[u_index]))
                    u_index +=1
                    
        s_i_error = []
        for i in range(0,len(x)):
            u_index = 0
            for u in s_i_atts.keys():
                if i == u:
                    s_i_error.append(np.abs(ground_truth[i] - list(s_i_atts.values())[u_index]))
                u_index +=1
                
        s_c_error = []
        for i in range(0,len(x)):
            u_index = 0
            for u in s_c_atts.keys():
                if i == u:
                    s_c_error.append(np.abs(ground_truth[i] - list(s_c_atts.values())[u_index]))
                u_index +=1
        overall_s_i.append(np.mean(s_i_error))
        overall_s_c.append(np.mean(s_c_error))
        overall_ss.append(np.mean(ss_error))
    return overall_s_i, overall_s_c, overall_ss
        

In [4]:
overall_s_i, overall_s_c, overall_ss = MAE()

0.9899065363591071
0
(1340, 6)
{(2,): array([0.07023394]), (4,): array([-0.04807581]), (0, 1): array([1.07521808]), (3, 5): array([-0.15717787])}
1
(1340, 6)
{(2,): array([0.10650854]), (3,): array([-0.51786511]), (0, 1): array([-0.15045645]), (4, 5): array([-0.15746562])}
2
(1340, 6)
{(2,): array([-0.06236968]), (3,): array([-0.21399908]), (0, 1): array([-1.09208192]), (4, 5): array([-0.70546751])}
3
(1340, 6)
{(0,): array([0.1944775]), (2,): array([-0.06236968]), (3,): array([0.29669273]), (4,): array([0.17048009]), (1, 5): array([0.7063367])}
4
(1340, 6)
{(2,): array([-0.00910482]), (3,): array([0.26005317]), (0, 1): array([-0.24056152]), (4, 5): array([-0.3913751])}
5
(1340, 6)
{(2,): array([-0.0406737]), (3,): array([-0.17595732]), (0, 1): array([-0.49750064]), (4, 5): array([-0.00051259])}
6
(1340, 6)
{(2,): array([0.00316809]), (3,): array([0.21952582]), (0, 1): array([-1.25864436]), (4, 5): array([-0.61273208])}
7
(1340, 6)
{(2,): array([0.00583455]), (3,): array([0.5811048]), 

In [5]:
print(np.mean(overall_s_i))
print(np.std(overall_s_i))`

0.11661637772410675
0.05179427835196047


In [6]:
print(np.mean(overall_s_c)) 
print (np.std(overall_s_c))

0.1573731467055543
0.0462309873880281


In [7]:
print(np.mean(overall_ss))
print(np.std(overall_ss))

0.15069177689315708
0.0666151904580166


In [8]:
np.std(overall_s_i)
np.std(overall_s_c)
np.std(overall_ss)

0.0666151904580166